In [1]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = "2NCD4p44jNbxqVXEFl5cEjcIg5fbqrNi5INs3VvPwQGPnt6zGWPKJQQJ99AJACmepeSXJ3w3AAABACOGAHUq"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://rag-chatapp.openai.azure.com/"

In [2]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4",
    api_version="2024-05-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # organization="...",
    # model="gpt-35-turbo",
    # model_version="0125",
)

In [3]:
messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]
llm.invoke(messages)

AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 28, 'total_tokens': 34, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'se

In [4]:
for chunk in llm.stream(messages):
    print(chunk)

content='' additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content='' additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content='J' additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content="'" additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content='ad' additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content='ore' additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content=' programmer' additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content='.' additional_kwargs={} response_metadata={} id='run-da0cb187-d64b-414d-905c-1c35572c6d52'
content='' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12'} id='run-da0cb187-d64b-414d-905c-1c

In [5]:
stream = llm.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
full

AIMessageChunk(content="J'adore programmer.", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12'}, id='run-c3a31d87-002a-4eeb-a20f-c5cacf80df02')

In [6]:
await llm.ainvoke(messages)

AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 28, 'total_tokens': 34, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'se

In [7]:
from typing import Optional

from pydantic import BaseModel, Field


class Joke(BaseModel):
    '''Joke to tell user.'''

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

KeyboardInterrupt: 

In [ ]:
Joke(
    setup="Why was the cat sitting on the computer?",
    punchline="To keep an eye on the mouse!",
    rating=None,
)

In [ ]:
json_llm = llm.bind(response_format={"type": "json_object"})
ai_msg = json_llm.invoke(
    "Return a JSON object with key 'random_ints' and a value of 10 random ints in [0-99]"
)
ai_msg.content

In [ ]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


llm_with_tools = llm.bind_tools([GetWeather, GetPopulation])
ai_msg = llm_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
ai_msg.tool_calls

In [ ]:
import base64
import httpx
from langchain_core.messages import HumanMessage

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content

In [ ]:
ai_msg = llm.invoke(messages)
ai_msg.usage_metadata

In [ ]:
logprobs_llm = llm.bind(logprobs=True)
ai_msg = logprobs_llm.invoke(messages)
ai_msg.response_metadata["logprobs"]

In [ ]:
import ssl
import certifi
import aiohttp

ssl_context = ssl.create_default_context(cafile=certifi.where())

async with aiohttp.ClientSession() as session:
    async with session.get('https://rag-chatapp.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-08-01-preview', ssl=ssl_context) as response:
        print(await response.text())

In [ ]:
import os
import sys
import hashlib
from pathlib import Path
import logging
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
import subprocess
import smtplib
from email.mime.text import MIMEText
import json
import time
from difflib import SequenceMatcher
from tqdm import tqdm
import send2trash
from PIL import Image, ExifTags
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from cryptography.fernet import Fernet
from plyer import notification

# GUI Imports
from PyQt5.QtWidgets import (
    QApplication, QWidget, QTabWidget, QVBoxLayout, QLabel, QPushButton,
    QFileDialog, QProgressBar, QTextEdit, QLineEdit, QHBoxLayout, QMessageBox,
    QComboBox, QListWidget, QListWidgetItem, QFormLayout, QSpinBox, QCheckBox,
    QGridLayout, QSizePolicy, QScrollArea
)
from PyQt5.QtGui import QPixmap, QIcon, QFont
from PyQt5.QtCore import Qt, QThread, pyqtSignal

# ===========================
# Ensure required packages are installed
# ===========================

required_packages = [
    'send2trash', 'Pillow', 'tqdm', 'cryptography', 'watchdog', 'PyQt5', 'plyer'
]

def install_missing_packages(required):
    import pkg_resources
    installed = {pkg.key for pkg in pkg_resources.working_set}
    missing = set(required) - installed
    if missing:
        print(f"Installing missing packages: {missing}")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])
    else:
        print("All required packages are already installed.")

try:
    import send2trash
    from PIL import Image, ExifTags
    from tqdm import tqdm
    from cryptography.fernet import Fernet
    from watchdog.observers import Observer
    from watchdog.events import FileSystemEventHandler
    from plyer import notification
except ImportError:
    install_missing_packages(required_packages)
    import send2trash
    from PIL import Image, ExifTags
    from tqdm import tqdm
    from cryptography.fernet import Fernet
    from watchdog.observers import Observer
    from watchdog.events import FileSystemEventHandler
    from plyer import notification

# ===========================
# Configuration and Constants
# ===========================

CONFIG_FILE = 'config.json'

def load_config():
    if not os.path.exists(CONFIG_FILE):
        default_config = {
            "file_categories": {
                "Images": [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"],
                "Videos": [".mp4", ".avi", ".mkv", ".mov", ".wmv"],
                "Documents": [".doc", ".docx", ".pdf", ".txt", ".xls", ".xlsx", ".ppt", ".pptx"],
                "Audio": [".mp3", ".wav", ".aac", ".flac"],
                "Archives": [".zip", ".rar", ".7z", ".tar", ".gz"],
                "Others": []
            },
            "backup_directory": "Backup",
            "encryption_key_file": "secret.key",
            "email_notifications": {
                "enabled": False,
                "smtp_server": "",
                "smtp_port": 587,
                "sender_email": "",
                "sender_password": "",
                "receiver_email": ""
            }
        }
        with open(CONFIG_FILE, 'w') as f:
            json.dump(default_config, f, indent=4)
        return default_config
    else:
        with open(CONFIG_FILE, 'r') as f:
            return json.load(f)

config = load_config()

# Setup logging
logging.basicConfig(
    filename='file_manager.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Define file type categories
FILE_CATEGORIES = config.get("file_categories", {
    'Images': ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'],
    'Videos': ['.mp4', '.avi', '.mkv', '.mov', '.wmv'],
    'Documents': ['.doc', '.docx', '.pdf', '.txt', '.xls', '.xlsx', '.ppt', '.pptx'],
    'Audio': ['.mp3', '.wav', '.aac', '.flac'],
    'Archives': ['.zip', '.rar', '.7z', '.tar', '.gz'],
    'Others': []
})

# ===========================
# Utility Functions
# ===========================

def send_email(subject, body):
    email_config = config.get("email_notifications", {})
    if not email_config.get("enabled"):
        return
    try:
        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = email_config['sender_email']
        msg['To'] = email_config['receiver_email']

        with smtplib.SMTP(email_config['smtp_server'], email_config['smtp_port']) as server:
            server.starttls()
            server.login(email_config['sender_email'], email_config['sender_password'])
            server.send_message(msg)
        logging.info("Email notification sent.")
    except Exception as e:
        logging.error(f"Failed to send email: {e}")

def generate_encryption_key():
    key = Fernet.generate_key()
    with open(config['encryption_key_file'], 'wb') as key_file:
        key_file.write(key)
    logging.info("Encryption key generated and saved.")
    return key

def load_encryption_key():
    key_file = config.get('encryption_key_file', 'secret.key')
    if not os.path.exists(key_file):
        return generate_encryption_key()
    with open(key_file, 'rb') as key_file:
        key = key_file.read()
    return key

ENCRYPTION_KEY = load_encryption_key()
fernet = Fernet(ENCRYPTION_KEY)

def backup_file(file_path, backup_dir):
    try:
        backup_dir = Path(backup_dir)
        backup_dir.mkdir(parents=True, exist_ok=True)
        relative_path = file_path.relative_to(file_path.parents[1])
        destination = backup_dir / relative_path
        destination.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(file_path, destination)
        logging.info(f"Backed up {file_path} to {destination}")
    except Exception as e:
        logging.error(f"Failed to backup {file_path}: {e}")

def calculate_sha256(file_path, block_size=65536):
    sha256 = hashlib.sha256()
    path = Path(file_path)
    try:
        if os.name == 'nt':
            path = Path(r'\\?\{}'.format(path))
        with path.open('rb') as f:
            for block in iter(lambda: f.read(block_size), b''):
                sha256.update(block)
    except Exception as e:
        logging.error(f"Couldn't process file {file_path} due to {e}")
        return None
    return sha256.hexdigest()

def group_files_by_size(directory):
    files_by_size = {}
    directory = Path(directory)
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            file_path = Path(dirpath) / filename
            try:
                if file_path.is_symlink():
                    continue
                file_size = file_path.stat().st_size
                files_by_size.setdefault(file_size, []).append(file_path)
            except Exception as e:
                logging.error(f"Error processing file {file_path}. Error: {e}")
    potential_duplicates = [files for files in files_by_size.values() if len(files) > 1]
    return potential_duplicates

def group_files_by_hash(potential_duplicates):
    duplicates = {}
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        future_to_file = {
            executor.submit(calculate_sha256, file_path): file_path
            for group in potential_duplicates for file_path in group
        }
        for future in tqdm(as_completed(future_to_file), total=len(future_to_file), desc="Hashing files", unit="file"):
            file_path = future_to_file[future]
            try:
                file_hash = future.result()
                if file_hash:
                    duplicates.setdefault(file_hash, []).append(file_path)
            except Exception as e:
                logging.error(f"Error hashing file {file_path}: {e}")
    duplicates = {hash_val: paths for hash_val, paths in duplicates.items() if len(paths) > 1}
    return duplicates

def find_duplicate_files(directory):
    logging.info(f"Starting duplicate search in: {directory}")
    potential_duplicates = group_files_by_size(directory)
    if not potential_duplicates:
        logging.info("No potential duplicates found based on file size.")
        return {}
    duplicates = group_files_by_hash(potential_duplicates)
    logging.info(f"Duplicate search completed. {len(duplicates)} sets of duplicates found.")
    return duplicates

def delete_files_permanently(duplicates, backup=True):
    logging.info("Starting permanent deletion of duplicates.")
    for file_hash, file_paths in duplicates.items():
        for file in file_paths[1:]:
            try:
                if backup:
                    backup_file(file, config.get("backup_directory", "Backup"))
                os.remove(file)
                logging.info(f"Permanently deleted: {file}")
                print(f"Deleted: {file}")
            except Exception as e:
                logging.error(f"Error deleting file {file}: {e}")
    print("Permanent deletion completed.")
    logging.info("Permanent deletion completed.")
    send_email("Duplicate Deletion Completed", "All duplicate files have been permanently deleted.")
    notification.notify(
        title="Deletion Completed",
        message="All duplicate files have been permanently deleted.",
        timeout=5
    )

def send_duplicates_to_trash(duplicates, backup=True):
    logging.info("Sending duplicates to trash.")
    for file_hash, file_paths in duplicates.items():
        for file in file_paths[1:]:
            try:
                if backup:
                    backup_file(file, config.get("backup_directory", "Backup"))
                send2trash.send2trash(str(file))
                logging.info(f"Sent to trash: {file}")
                print(f"Sent to trash: {file}")
            except Exception as e:
                logging.error(f"Error sending file {file} to trash: {e}")
    print("Duplicates sent to trash.")
    logging.info("Duplicates sent to trash.")
    send_email("Duplicate Trash Completed", "All duplicate files have been moved to trash.")
    notification.notify(
        title="Trash Operation Completed",
        message="All duplicate files have been moved to trash.",
        timeout=5
    )

def sort_files(directory, target_directory=None, sort_by='type'):
    if target_directory is None:
        target_directory = Path(directory) / "Sorted_Files"
    target_directory = Path(target_directory)
    target_directory.mkdir(exist_ok=True)
    
    logging.info(f"Starting to sort files in: {directory} by {sort_by}")
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            file_path = Path(dirpath) / filename
            try:
                if file_path.is_symlink():
                    continue
                if file_path.parent == target_directory:
                    continue
                if sort_by == 'type':
                    file_ext = file_path.suffix.lower()
                    category_found = False
                    for category, extensions in FILE_CATEGORIES.items():
                        if file_ext in extensions:
                            category_dir = target_directory / category
                            category_dir.mkdir(exist_ok=True)
                            shutil.move(str(file_path), category_dir / file_path.name)
                            logging.info(f"Moved {file_path} to {category_dir}")
                            category_found = True
                            break
                    if not category_found:
                        others_dir = target_directory / "Others"
                        others_dir.mkdir(exist_ok=True)
                        shutil.move(str(file_path), others_dir / file_path.name)
                        logging.info(f"Moved {file_path} to {others_dir}")
                elif sort_by == 'date':
                    creation_time = file_path.stat().st_ctime
                    date_folder = time.strftime('%Y-%m-%d', time.localtime(creation_time))
                    date_dir = target_directory / date_folder
                    date_dir.mkdir(exist_ok=True)
                    shutil.move(str(file_path), date_dir / file_path.name)
                    logging.info(f"Moved {file_path} to {date_dir}")
            except Exception as e:
                logging.error(f"Error sorting file {file_path}: {e}")
    print(f"Files have been sorted and moved to {target_directory}")
    logging.info(f"File sorting completed. Sorted files are in {target_directory}")
    send_email("File Sorting Completed", f"Files in {directory} have been sorted by {sort_by}.")
    notification.notify(
        title="File Sorting Completed",
        message=f"Files in {directory} have been sorted by {sort_by}.",
        timeout=5
    )

def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = Path(dirpath) / f
            if fp.is_file():
                total_size += fp.stat().st_size
    return total_size

def directory_size_report(directory, report_path="directory_size_report.txt"):
    try:
        sizes = {}
        for item in os.listdir(directory):
            item_path = Path(directory) / item
            if item_path.is_dir():
                sizes[item] = get_directory_size(item_path)
            else:
                sizes[item] = item_path.stat().st_size
        sorted_sizes = sorted(sizes.items(), key=lambda x: x[1], reverse=True)
        with open(report_path, 'w') as report_file:
            report_file.write(f"Directory Size Report - {Path(report_path).parent.resolve()}\n")
            report_file.write("="*50 + "\n\n")
            for name, size in sorted_sizes:
                readable_size = convert_size(size)
                report_file.write(f"{name}: {readable_size}\n")
        print(f"Directory size report generated at {report_path}\n")
        logging.info(f"Directory size report generated at {report_path}")
        notification.notify(
            title="Report Generated",
            message=f"Directory size report generated at {report_path}",
            timeout=5
        )
    except Exception as e:
        logging.error(f"Error generating directory size report: {e}")
        print(f"Failed to generate directory size report: {e}")
        notification.notify(
            title="Report Generation Failed",
            message=f"Failed to generate directory size report: {e}",
            timeout=5
        )

def convert_size(size_bytes):
    # Converts bytes to a more readable format
    for unit in ['B','KB','MB','GB','TB']:
        if size_bytes < 1024:
            return f"{size_bytes:.2f}{unit}"
        size_bytes /= 1024
    return f"{size_bytes:.2f}PB"

def is_similar(file1, file2, threshold=0.9):
    """Check if two text files are similar based on SequenceMatcher."""
    try:
        with open(file1, 'r', errors='ignore') as f1, open(file2, 'r', errors='ignore') as f2:
            text1 = f1.read()
            text2 = f2.read()
            similarity = SequenceMatcher(None, text1, text2).ratio()
            return similarity >= threshold
    except:
        return False

def fuzzy_duplicate_detection(duplicates):
    # Placeholder for fuzzy duplicate detection
    # Implementing this requires domain-specific logic
    # For example, for images, you might use perceptual hashing
    # Here, we simply return the original duplicates
    return duplicates

def encrypt_file(file_path, encrypted_path):
    try:
        with open(file_path, 'rb') as f:
            data = f.read()
        encrypted_data = fernet.encrypt(data)
        with open(encrypted_path, 'wb') as f:
            f.write(encrypted_data)
        logging.info(f"Encrypted {file_path} to {encrypted_path}")
        print(f"Encrypted: {file_path} -> {encrypted_path}")
        notification.notify(
            title="Encryption Completed",
            message=f"Encrypted: {file_path} -> {encrypted_path}",
            timeout=5
        )
    except Exception as e:
        logging.error(f"Failed to encrypt {file_path}: {e}")
        print(f"Failed to encrypt {file_path}: {e}")
        notification.notify(
            title="Encryption Failed",
            message=f"Failed to encrypt {file_path}: {e}",
            timeout=5
        )

def decrypt_file(encrypted_path, decrypted_path):
    try:
        with open(encrypted_path, 'rb') as f:
            data = f.read()
        decrypted_data = fernet.decrypt(data)
        with open(decrypted_path, 'wb') as f:
            f.write(decrypted_data)
        logging.info(f"Decrypted {encrypted_path} to {decrypted_path}")
        print(f"Decrypted: {encrypted_path} -> {decrypted_path}")
        notification.notify(
            title="Decryption Completed",
            message=f"Decrypted: {encrypted_path} -> {decrypted_path}",
            timeout=5
        )
    except Exception as e:
        logging.error(f"Failed to decrypt {encrypted_path}: {e}")
        print(f"Failed to decrypt {encrypted_path}: {e}")
        notification.notify(
            title="Decryption Failed",
            message=f"Failed to decrypt {encrypted_path}: {e}",
            timeout=5
        )

def display_image_metadata(image_path):
    try:
        with Image.open(image_path) as img:
            info = img._getexif()
            if not info:
                logging.info(f"No metadata found for {image_path}")
                return "No metadata found."
            metadata = ""
            for tag, value in info.items():
                tag_name = ExifTags.TAGS.get(tag, tag)
                metadata += f"{tag_name}: {value}\n"
            logging.info(f"Displayed metadata for {image_path}")
            return metadata
    except Exception as e:
        logging.error(f"Failed to read metadata from {image_path}: {e}")
        print(f"Failed to read metadata from {image_path}: {e}")
        return f"Failed to read metadata from {image_path}: {e}"

def remove_image_metadata(image_path, output_path):
    try:
        with Image.open(image_path) as img:
            data = list(img.getdata())
            img_without_exif = Image.new(img.mode, img.size)
            img_without_exif.putdata(data)
            img_without_exif.save(output_path)
        logging.info(f"Removed metadata from {image_path} and saved to {output_path}")
        print(f"Removed metadata: {image_path} -> {output_path}")
        notification.notify(
            title="Metadata Removal Completed",
            message=f"Removed metadata: {image_path} -> {output_path}",
            timeout=5
        )
    except Exception as e:
        logging.error(f"Failed to remove metadata from {image_path}: {e}")
        print(f"Failed to remove metadata from {image_path}: {e}")
        notification.notify(
            title="Metadata Removal Failed",
            message=f"Failed to remove metadata from {image_path}: {e}",
            timeout=5
        )

def batch_rename_files(directory, pattern, start_index=1):
    try:
        files = sorted([f for f in Path(directory).iterdir() if f.is_file()])
        for i, file in enumerate(files, start=start_index):
            new_name = pattern.format(index=i, original_name=file.stem, extension=file.suffix)
            new_path = file.parent / new_name
            shutil.move(str(file), new_path)
            logging.info(f"Renamed {file} to {new_path}")
            print(f"Renamed: {file} -> {new_path}")
        print("Batch renaming completed.")
        logging.info("Batch renaming completed.")
        notification.notify(
            title="Batch Renaming Completed",
            message="All selected files have been renamed.",
            timeout=5
        )
    except Exception as e:
        logging.error(f"Failed to batch rename files in {directory}: {e}")
        print(f"Failed to batch rename files in {directory}: {e}")
        notification.notify(
            title="Batch Renaming Failed",
            message=f"Failed to batch rename files in {directory}: {e}",
            timeout=5
        )

class Watcher:
    def __init__(self, directory_to_watch):
        self.DIRECTORY_TO_WATCH = directory_to_watch
        self.observer = Observer()

    def run(self):
        event_handler = Handler()
        self.observer.schedule(event_handler, self.DIRECTORY_TO_WATCH, recursive=True)
        self.observer.start()
        logging.info(f"Started real-time monitoring on {self.DIRECTORY_TO_WATCH}")
        print(f"Real-time monitoring started on {self.DIRECTORY_TO_WATCH}. Press Ctrl+C to stop.")
        notification.notify(
            title="Monitoring Started",
            message=f"Real-time monitoring started on {self.DIRECTORY_TO_WATCH}.",
            timeout=5
        )
        try:
            while True:
                time.sleep(5)
        except KeyboardInterrupt:
            self.observer.stop()
            logging.info("Stopped real-time monitoring.")
            print("\nReal-time monitoring stopped.")
            notification.notify(
                title="Monitoring Stopped",
                message=f"Real-time monitoring stopped on {self.DIRECTORY_TO_WATCH}.",
                timeout=5
            )
        self.observer.join()

class Handler(FileSystemEventHandler):
    @staticmethod
    def on_created(event):
        if event.is_directory:
            return
        logging.info(f"Detected new file: {event.src_path}")
        print(f"New file detected: {event.src_path}")
        send_email("New File Detected", f"A new file has been created: {event.src_path}")
        notification.notify(
            title="New File Detected",
            message=f"A new file has been created: {event.src_path}",
            timeout=5
        )

def generate_log_summary(log_path='file_manager.log', summary_path='log_summary.txt'):
    try:
        if not os.path.exists(log_path):
            print("Log file does not exist.")
            return
        summary = {}
        with open(log_path, 'r') as log_file:
            for line in log_file:
                parts = line.split(' - ')
                if len(parts) < 3:
                    continue
                _, level, _ = parts
                summary[level] = summary.get(level, 0) + 1
        with open(summary_path, 'w') as summary_file:
            summary_file.write("Log Summary\n")
            summary_file.write("="*50 + "\n\n")
            for level, count in summary.items():
                summary_file.write(f"{level}: {count}\n")
        print(f"Log summary generated at {summary_path}")
        logging.info(f"Log summary generated at {summary_path}")
        notification.notify(
            title="Log Summary Generated",
            message=f"Log summary generated at {summary_path}",
            timeout=5
        )
    except Exception as e:
        logging.error(f"Failed to generate log summary: {e}")
        print(f"Failed to generate log summary: {e}")
        notification.notify(
            title="Log Summary Failed",
            message=f"Failed to generate log summary: {e}",
            timeout=5
        )

def compress_images(input_folder, output_folder, quality=85):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(('jpg', 'jpeg', 'png', 'webp')):
                input_path = os.path.join(root, file)
                output_path = os.path.join(output_folder, file)
                try:
                    with Image.open(input_path) as img:
                        if img.mode in ("RGBA", "P"):
                            img = img.convert("RGB")
                        img.save(output_path, optimize=True, quality=quality)
                        print(f"Compressed: {input_path} -> {output_path}")
                        logging.info(f"Compressed: {input_path} -> {output_path}")
                except Exception as e:
                    logging.error(f"Failed to compress {input_path}: {e}")
    notification.notify(
        title="Image Compression Completed",
        message=f"Images compressed and saved to {output_folder}",
        timeout=5
    )

def convert_images(input_folder, output_folder, target_format):
    target_format = target_format.lower()
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    supported_formats = ('jpg', 'jpeg', 'png', 'webp')
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(supported_formats):
                input_path = os.path.join(root, file)
                output_filename = os.path.splitext(file)[0] + f".{target_format}"
                output_path = os.path.join(output_folder, output_filename)
                try:
                    with Image.open(input_path) as img:
                        if target_format in ("jpg", "jpeg") and img.mode in ("RGBA", "P"):
                            img = img.convert("RGB")
                        img.save(output_path, format=target_format.upper())
                        print(f"Converted: {input_path} -> {output_path}")
                        logging.info(f"Converted: {input_path} -> {output_path}")
                except Exception as e:
                    logging.error(f"Failed to convert {input_path}: {e}")
    notification.notify(
        title="Image Conversion Completed",
        message=f"Images converted to {target_format.upper()} and saved to {output_folder}",
        timeout=5
    )

def generate_report(duplicates, report_path="duplicate_report.txt"):
    try:
        with open(report_path, 'w') as report_file:
            if not duplicates:
                report_file.write("No duplicates found.\n")
                print("No duplicates found. Report generated.")
                logging.info("No duplicates found. Report generated.")
                notification.notify(
                    title="Duplicate Report",
                    message="No duplicates found.",
                    timeout=5
                )
                return
            report_file.write(f"Duplicate Files Report - {Path(report_path).parent.resolve()}\n")
            report_file.write("="*50 + "\n\n")
            for file_hash, file_paths in duplicates.items():
                report_file.write(f"Hash: {file_hash}\n")
                for path in file_paths:
                    report_file.write(f"    {path}\n")
                report_file.write("\n")
        print(f"Report generated at {report_path}\n")
        logging.info(f"Duplicate report generated at {report_path}")
        notification.notify(
            title="Duplicate Report Generated",
            message=f"Duplicate report generated at {report_path}",
            timeout=5
        )
    except Exception as e:
        logging.error(f"Error generating report: {e}")
        print(f"Failed to generate report: {e}")
        notification.notify(
            title="Report Generation Failed",
            message=f"Failed to generate report: {e}",
            timeout=5
        )

# ===========================
# GUI Components
# ===========================

class DuplicateFinderThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal(dict)
    def __init__(self, directory):
        super().__init__()
        self.directory = directory

    def run(self):
        duplicates = find_duplicate_files(self.directory)
        self.finished.emit(duplicates)

class SortFilesThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal()
    def __init__(self, directory, sort_by, target_directory):
        super().__init__()
        self.directory = directory
        self.sort_by = sort_by
        self.target_directory = target_directory

    def run(self):
        sort_files(self.directory, sort_by=self.sort_by, target_directory=self.target_directory)
        self.finished.emit()

class CompressImagesThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal()
    def __init__(self, input_folder, output_folder, quality):
        super().__init__()
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.quality = quality

    def run(self):
        compress_images(self.input_folder, self.output_folder, self.quality)
        self.finished.emit()

class ConvertImagesThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal()
    def __init__(self, input_folder, output_folder, target_format):
        super().__init__()
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.target_format = target_format

    def run(self):
        convert_images(self.input_folder, self.output_folder, self.target_format)
        self.finished.emit()

class EncryptionThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal(str)
    def __init__(self, action, file_path, output_path):
        super().__init__()
        self.action = action
        self.file_path = file_path
        self.output_path = output_path

    def run(self):
        if self.action == 'encrypt':
            encrypt_file(self.file_path, self.output_path)
            self.finished.emit("Encryption Completed")
        elif self.action == 'decrypt':
            decrypt_file(self.file_path, self.output_path)
            self.finished.emit("Decryption Completed")

class MetadataThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal(str)
    def __init__(self, action, image_path, output_path=None):
        super().__init__()
        self.action = action
        self.image_path = image_path
        self.output_path = output_path

    def run(self):
        if self.action == 'display':
            metadata = display_image_metadata(self.image_path)
            self.finished.emit(metadata)
        elif self.action == 'remove':
            remove_image_metadata(self.image_path, self.output_path)
            self.finished.emit("Metadata Removal Completed")

class BatchRenameThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal()
    def __init__(self, directory, pattern, start_index):
        super().__init__()
        self.directory = directory
        self.pattern = pattern
        self.start_index = start_index

    def run(self):
        batch_rename_files(self.directory, self.pattern, self.start_index)
        self.finished.emit()

class DirectorySizeReportThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal()
    def __init__(self, directory, report_path):
        super().__init__()
        self.directory = directory
        self.report_path = report_path

    def run(self):
        directory_size_report(self.directory, self.report_path)
        self.finished.emit()

class LogSummaryThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal()
    def __init__(self, log_path, summary_path):
        super().__init__()
        self.log_path = log_path
        self.summary_path = summary_path

    def run(self):
        generate_log_summary(self.log_path, self.summary_path)
        self.finished.emit()

class CacheCleanerThread(QThread):
    progress = pyqtSignal(int)
    finished = pyqtSignal()
    def __init__(self, directories):
        super().__init__()
        self.directories = directories

    def run(self):
        try:
            total_dirs = len(self.directories)
            for idx, directory in enumerate(self.directories, 1):
                self.progress.emit(int((idx / total_dirs) * 100))
                clean_directory(directory)
            self.finished.emit()
        except Exception as e:
            logging.error(f"Error during cache cleaning: {e}")
            print(f"Failed to clean cache directories: {e}")
            notification.notify(
                title="Cache Cleaning Failed",
                message=f"Failed to clean cache directories: {e}",
                timeout=5
            )

def clean_directory(directory):
    if not os.path.exists(directory):
        logging.warning(f"Directory does not exist: {directory}")
        return
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = Path(root) / file
            try:
                if file_path.is_file():
                    file_path.unlink()
                    logging.info(f"Deleted cache file: {file_path}")
            except Exception as e:
                logging.error(f"Failed to delete {file_path}: {e}")

# ===========================
# GUI Components Continued
# ===========================

class FileManagerGUI(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("File Manager Pro")
        self.setGeometry(100, 100, 1000, 800)
        self.setWindowIcon(QIcon('icon.png'))  # Optional: Add an icon.png in the script directory
        self.layout = QVBoxLayout()
        self.tabs = QTabWidget()
        self.layout.addWidget(self.tabs)
        self.setLayout(self.layout)

        # Initialize all tabs
        self.init_duplicate_finder_tab()
        self.init_file_sorter_tab()
        self.init_image_tools_tab()
        self.init_encryption_tab()
        self.init_metadata_tab()
        self.init_batch_rename_tab()
        self.init_reports_tab()
        self.init_monitor_tab()
        self.init_cache_cleaner_tab()
        self.init_settings_tab()

        # Apply Stylesheet
        self.apply_stylesheet()

    def apply_stylesheet(self):
        self.setStyleSheet("""
            QWidget {
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                font-size: 14px;
            }
            QPushButton {
                background-color: #007ACC;
                color: white;
                padding: 10px 20px;
                border: none;
                border-radius: 5px;
                font-size: 14px;
            }
            QPushButton:hover {
                background-color: #005A9E;
            }
            QLineEdit, QTextEdit, QComboBox, QSpinBox, QListWidget {
                padding: 8px;
                border: 1px solid #ccc;
                border-radius: 4px;
                font-size: 14px;
            }
            QTabWidget::pane { /* The tab widget frame */
                border-top: 2px solid #C2C7CB;
            }
            QTabBar::tab {
                background: #f1f1f1;
                border: 1px solid #C4C4C3;
                padding: 12px;
                border-bottom: none;
                min-width: 120px;
                font-weight: bold;
            }
            QTabBar::tab:selected {
                background: #ffffff;
                border-bottom: 2px solid #007ACC;
            }
            QLabel {
                padding: 5px;
                font-size: 14px;
            }
            QProgressBar {
                border: 1px solid #ccc;
                border-radius: 5px;
                text-align: center;
            }
            QProgressBar::chunk {
                background-color: #007ACC;
                width: 20px;
            }
            QScrollArea {
                background: #ffffff;
            }
        """)

    def init_duplicate_finder_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Find and Handle Duplicate Files")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        # Select Directory Button and Label
        dir_layout = QHBoxLayout()
        self.df_select_dir_btn = QPushButton("📂 Select Directory")
        self.df_select_dir_btn.setToolTip("Choose the directory to scan for duplicate files.")
        self.df_select_dir_btn.clicked.connect(self.select_duplicate_directory)
        dir_layout.addWidget(self.df_select_dir_btn)

        self.df_directory_label = QLabel("No directory selected.")
        self.df_directory_label.setStyleSheet("color: #555555;")
        dir_layout.addWidget(self.df_directory_label)

        layout.addLayout(dir_layout)

        # Find Duplicates Button
        self.df_find_btn = QPushButton("🔍 Find Duplicates")
        self.df_find_btn.setToolTip("Start scanning the selected directory for duplicate files.")
        self.df_find_btn.clicked.connect(self.find_duplicates)
        self.df_find_btn.setEnabled(False)
        layout.addWidget(self.df_find_btn)

        # Progress Bar
        self.df_progress = QProgressBar()
        layout.addWidget(self.df_progress)

        # Results Display
        self.df_results = QTextEdit()
        self.df_results.setReadOnly(True)
        self.df_results.setPlaceholderText("Duplicate files will be listed here.")
        layout.addWidget(self.df_results)

        # Handle Duplicates Buttons
        handle_layout = QHBoxLayout()
        self.df_delete_btn = QPushButton("🗑️ Permanently Delete Duplicates")
        self.df_delete_btn.setToolTip("Delete all duplicate files permanently.")
        self.df_delete_btn.clicked.connect(self.delete_duplicates)
        self.df_delete_btn.setEnabled(False)
        handle_layout.addWidget(self.df_delete_btn)

        self.df_trash_btn = QPushButton("🗄️ Move Duplicates to Trash")
        self.df_trash_btn.setToolTip("Move all duplicate files to the system trash.")
        self.df_trash_btn.clicked.connect(self.trash_duplicates)
        self.df_trash_btn.setEnabled(False)
        handle_layout.addWidget(self.df_trash_btn)

        layout.addLayout(handle_layout)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Duplicate Finder")

    def select_duplicate_directory(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Directory")
        if directory:
            self.df_directory_label.setText(directory)
            self.df_find_btn.setEnabled(True)
            self.selected_duplicate_directory = directory

    def find_duplicates(self):
        self.df_results.clear()
        self.df_find_btn.setEnabled(False)
        self.df_delete_btn.setEnabled(False)
        self.df_trash_btn.setEnabled(False)
        self.df_progress.setValue(0)
        self.duplicate_thread = DuplicateFinderThread(self.selected_duplicate_directory)
        self.duplicate_thread.finished.connect(self.display_duplicates)
        self.duplicate_thread.start()

    def display_duplicates(self, duplicates):
        if not duplicates:
            self.df_results.setText("No duplicates found.")
            self.df_delete_btn.setEnabled(False)
            self.df_trash_btn.setEnabled(False)
            self.df_progress.setValue(100)
            return
        report = f"Found {len(duplicates)} sets of duplicates:\n\n"
        for idx, (hash_val, paths) in enumerate(duplicates.items(), 1):
            report += f"Set {idx} - Hash: {hash_val}\n"
            for path in paths:
                report += f"    {path}\n"
            report += "\n"
        self.df_results.setText(report)
        self.current_duplicates = duplicates
        self.df_delete_btn.setEnabled(True)
        self.df_trash_btn.setEnabled(True)
        self.df_progress.setValue(100)
        generate_report(duplicates)

    def delete_duplicates(self):
        confirm = QMessageBox.question(self, "Confirm Deletion",
                                       "Are you sure you want to permanently delete the duplicates?\nThis action cannot be undone.",
                                       QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if confirm == QMessageBox.Yes:
            delete_files_permanently(self.current_duplicates)

    def trash_duplicates(self):
        confirm = QMessageBox.question(self, "Confirm Trash",
                                       "Are you sure you want to move the duplicates to trash?",
                                       QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if confirm == QMessageBox.Yes:
            send_duplicates_to_trash(self.current_duplicates)

    def init_file_sorter_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Sort Files by Type or Date")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        # Select Directory Button and Label
        dir_layout = QHBoxLayout()
        self.fs_select_dir_btn = QPushButton("📂 Select Directory to Sort")
        self.fs_select_dir_btn.setToolTip("Choose the directory containing files to sort.")
        self.fs_select_dir_btn.clicked.connect(self.select_sort_directory)
        dir_layout.addWidget(self.fs_select_dir_btn)

        self.fs_directory_label = QLabel("No directory selected.")
        self.fs_directory_label.setStyleSheet("color: #555555;")
        dir_layout.addWidget(self.fs_directory_label)

        layout.addLayout(dir_layout)

        # Sort By Dropdown
        sort_layout = QHBoxLayout()
        sort_label = QLabel("Sort By:")
        sort_layout.addWidget(sort_label)

        self.fs_sort_by = QComboBox()
        self.fs_sort_by.addItems(["Type", "Date"])
        self.fs_sort_by.setToolTip("Choose whether to sort files by their type or creation date.")
        sort_layout.addWidget(self.fs_sort_by)

        layout.addLayout(sort_layout)

        # Select Target Directory Button and Label
        target_layout = QHBoxLayout()
        self.fs_select_target_btn = QPushButton("📁 Select Target Directory (Optional)")
        self.fs_select_target_btn.setToolTip("Choose where sorted files will be moved. If not selected, a 'Sorted_Files' folder will be created.")
        self.fs_select_target_btn.clicked.connect(self.select_sort_target_directory)
        target_layout.addWidget(self.fs_select_target_btn)

        self.fs_target_label = QLabel("No target directory selected.")
        self.fs_target_label.setStyleSheet("color: #555555;")
        target_layout.addWidget(self.fs_target_label)

        layout.addLayout(target_layout)

        # Sort Files Button
        self.fs_sort_btn = QPushButton("📑 Sort Files")
        self.fs_sort_btn.setToolTip("Start sorting the selected files based on the chosen criteria.")
        self.fs_sort_btn.clicked.connect(self.sort_files_action)
        self.fs_sort_btn.setEnabled(False)
        layout.addWidget(self.fs_sort_btn)

        # Progress Bar
        self.fs_progress = QProgressBar()
        layout.addWidget(self.fs_progress)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "File Sorter")

    def select_sort_directory(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Directory to Sort")
        if directory:
            self.fs_directory_label.setText(directory)
            self.fs_sort_btn.setEnabled(True)
            self.selected_sort_directory = directory

    def select_sort_target_directory(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Target Directory")
        if directory:
            self.fs_target_label.setText(directory)
            self.selected_sort_target_directory = directory

    def sort_files_action(self):
        sort_by = self.fs_sort_by.currentText().lower()
        target_directory = getattr(self, 'selected_sort_target_directory', None)
        self.fs_sort_btn.setEnabled(False)
        self.fs_progress.setValue(0)
        self.sort_thread = SortFilesThread(self.selected_sort_directory, sort_by=sort_by, target_directory=target_directory)
        self.sort_thread.finished.connect(self.sort_completed)
        self.sort_thread.start()

    def sort_completed(self):
        self.fs_sort_btn.setEnabled(True)
        self.fs_progress.setValue(100)

    def init_image_tools_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Image Compression and Conversion")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        # Compression Section
        comp_section = QWidget()
        comp_layout = QVBoxLayout()
        comp_label = QLabel("📦 Compress Images")
        comp_label.setStyleSheet("font-size: 16px; font-weight: bold;")
        comp_layout.addWidget(comp_label)

        # Select Input Folder
        comp_input_layout = QHBoxLayout()
        self.it_compress_select_input_btn = QPushButton("📂 Select Input Folder")
        self.it_compress_select_input_btn.setToolTip("Choose the folder containing images to compress.")
        self.it_compress_select_input_btn.clicked.connect(self.select_compress_input_folder)
        comp_input_layout.addWidget(self.it_compress_select_input_btn)

        self.it_compress_input_label = QLabel("No input folder selected.")
        self.it_compress_input_label.setStyleSheet("color: #555555;")
        comp_input_layout.addWidget(self.it_compress_input_label)

        comp_layout.addLayout(comp_input_layout)

        # Select Output Folder
        comp_output_layout = QHBoxLayout()
        self.it_compress_select_output_btn = QPushButton("📁 Select Output Folder")
        self.it_compress_select_output_btn.setToolTip("Choose where compressed images will be saved.")
        self.it_compress_select_output_btn.clicked.connect(self.select_compress_output_folder)
        comp_output_layout.addWidget(self.it_compress_select_output_btn)

        self.it_compress_output_label = QLabel("No output folder selected.")
        self.it_compress_output_label.setStyleSheet("color: #555555;")
        comp_output_layout.addWidget(self.it_compress_output_label)

        comp_layout.addLayout(comp_output_layout)

        # Compression Quality
        quality_layout = QHBoxLayout()
        self.it_compress_quality = QSpinBox()
        self.it_compress_quality.setRange(1, 100)
        self.it_compress_quality.setValue(85)
        self.it_compress_quality.setToolTip("Set the compression quality (1-100). Higher values retain more quality.")
        quality_label = QLabel("Compression Quality:")
        quality_layout.addWidget(quality_label)
        quality_layout.addWidget(self.it_compress_quality)
        comp_layout.addLayout(quality_layout)

        # Compress Button
        self.it_compress_btn = QPushButton("🔧 Compress Images")
        self.it_compress_btn.setToolTip("Start compressing the selected images.")
        self.it_compress_btn.clicked.connect(self.compress_images_action)
        self.it_compress_btn.setEnabled(False)
        comp_layout.addWidget(self.it_compress_btn)

        # Add Compression Layout
        comp_section.setLayout(comp_layout)
        layout.addWidget(comp_section)

        # Conversion Section
        conv_section = QWidget()
        conv_layout = QVBoxLayout()
        conv_label = QLabel("🔄 Convert Images")
        conv_label.setStyleSheet("font-size: 16px; font-weight: bold;")
        conv_layout.addWidget(conv_label)

        # Select Input Folder
        conv_input_layout = QHBoxLayout()
        self.it_convert_select_input_btn = QPushButton("📂 Select Input Folder")
        self.it_convert_select_input_btn.setToolTip("Choose the folder containing images to convert.")
        self.it_convert_select_input_btn.clicked.connect(self.select_convert_input_folder)
        conv_input_layout.addWidget(self.it_convert_select_input_btn)

        self.it_convert_input_label = QLabel("No input folder selected.")
        self.it_convert_input_label.setStyleSheet("color: #555555;")
        conv_input_layout.addWidget(self.it_convert_input_label)

        conv_layout.addLayout(conv_input_layout)

        # Select Output Folder
        conv_output_layout = QHBoxLayout()
        self.it_convert_select_output_btn = QPushButton("📁 Select Output Folder")
        self.it_convert_select_output_btn.setToolTip("Choose where converted images will be saved.")
        self.it_convert_select_output_btn.clicked.connect(self.select_convert_output_folder)
        conv_output_layout.addWidget(self.it_convert_select_output_btn)

        self.it_convert_output_label = QLabel("No output folder selected.")
        self.it_convert_output_label.setStyleSheet("color: #555555;")
        conv_output_layout.addWidget(self.it_convert_output_label)

        conv_layout.addLayout(conv_output_layout)

        # Select Target Format
        format_layout = QHBoxLayout()
        self.it_convert_format = QComboBox()
        self.it_convert_format.addItems(["png", "jpg", "webp"])
        self.it_convert_format.setToolTip("Choose the desired format to convert images to.")
        format_label = QLabel("Target Format:")
        format_layout.addWidget(format_label)
        format_layout.addWidget(self.it_convert_format)
        conv_layout.addLayout(format_layout)

        # Convert Button
        self.it_convert_btn = QPushButton("🔧 Convert Images")
        self.it_convert_btn.setToolTip("Start converting the selected images.")
        self.it_convert_btn.clicked.connect(self.convert_images_action)
        self.it_convert_btn.setEnabled(False)
        conv_layout.addWidget(self.it_convert_btn)

        # Add Conversion Layout
        conv_section.setLayout(conv_layout)
        layout.addWidget(conv_section)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Image Tools")

    def select_compress_input_folder(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Input Folder for Compression")
        if directory:
            self.it_compress_input_label.setText(directory)
            self.selected_compress_input = directory
            self.check_compress_ready()

    def select_compress_output_folder(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Output Folder for Compression")
        if directory:
            self.it_compress_output_label.setText(directory)
            self.selected_compress_output = directory
            self.check_compress_ready()

    def check_compress_ready(self):
        if hasattr(self, 'selected_compress_input') and hasattr(self, 'selected_compress_output'):
            self.it_compress_btn.setEnabled(True)

    def compress_images_action(self):
        quality = self.it_compress_quality.value()
        self.it_compress_btn.setEnabled(False)
        self.compress_thread = CompressImagesThread(self.selected_compress_input, self.selected_compress_output, quality)
        self.compress_thread.finished.connect(self.compress_completed)
        self.compress_thread.start()

    def compress_completed(self):
        self.it_compress_btn.setEnabled(True)

    def select_convert_input_folder(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Input Folder for Conversion")
        if directory:
            self.it_convert_input_label.setText(directory)
            self.selected_convert_input = directory
            self.check_convert_ready()

    def select_convert_output_folder(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Output Folder for Conversion")
        if directory:
            self.it_convert_output_label.setText(directory)
            self.selected_convert_output = directory
            self.check_convert_ready()

    def check_convert_ready(self):
        if hasattr(self, 'selected_convert_input') and hasattr(self, 'selected_convert_output'):
            self.it_convert_btn.setEnabled(True)

    def convert_images_action(self):
        target_format = self.it_convert_format.currentText()
        self.it_convert_btn.setEnabled(False)
        self.convert_thread = ConvertImagesThread(self.selected_convert_input, self.selected_convert_output, target_format)
        self.convert_thread.finished.connect(self.convert_completed)
        self.convert_thread.start()

    def convert_completed(self):
        self.it_convert_btn.setEnabled(True)

    def init_encryption_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Encrypt and Decrypt Files")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        form_layout = QFormLayout()

        # Action Selection
        self.et_action = QComboBox()
        self.et_action.addItems(["Encrypt", "Decrypt"])
        self.et_action.setToolTip("Choose whether to encrypt or decrypt a file.")
        form_layout.addRow("Action:", self.et_action)

        # Select File
        self.et_select_file_btn = QPushButton("📂 Select File")
        self.et_select_file_btn.setToolTip("Choose the file to encrypt or decrypt.")
        self.et_select_file_btn.clicked.connect(self.select_encrypt_file)
        form_layout.addRow("File:", self.et_select_file_btn)

        self.et_file_label = QLabel("No file selected.")
        self.et_file_label.setStyleSheet("color: #555555;")
        form_layout.addRow("", self.et_file_label)

        # Select Output Path
        self.et_output_btn = QPushButton("📁 Select Output Path")
        self.et_output_btn.setToolTip("Choose where the encrypted/decrypted file will be saved.")
        self.et_output_btn.clicked.connect(self.select_encrypt_output)
        form_layout.addRow("Output Path:", self.et_output_btn)

        self.et_output_label = QLabel("No output path selected.")
        self.et_output_label.setStyleSheet("color: #555555;")
        form_layout.addRow("", self.et_output_label)

        layout.addLayout(form_layout)

        # Encrypt/Decrypt Button
        self.et_encrypt_btn = QPushButton("🔒 Start")
        self.et_encrypt_btn.setToolTip("Start the encryption or decryption process.")
        self.et_encrypt_btn.clicked.connect(self.encryption_action)
        self.et_encrypt_btn.setEnabled(False)
        layout.addWidget(self.et_encrypt_btn)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Encryption")

    def select_encrypt_file(self):
        file, _ = QFileDialog.getOpenFileName(self, "Select File")
        if file:
            self.et_file_label.setText(file)
            self.selected_encrypt_file = file
            self.check_encryption_ready()

    def select_encrypt_output(self):
        file, _ = QFileDialog.getSaveFileName(self, "Select Output File")
        if file:
            self.et_output_label.setText(file)
            self.selected_encrypt_output = file
            self.check_encryption_ready()

    def check_encryption_ready(self):
        if hasattr(self, 'selected_encrypt_file') and hasattr(self, 'selected_encrypt_output'):
            self.et_encrypt_btn.setEnabled(True)

    def encryption_action(self):
        action = self.et_action.currentText().lower()
        file_path = self.selected_encrypt_file
        output_path = self.selected_encrypt_output
        self.et_encrypt_btn.setEnabled(False)
        self.encryption_thread = EncryptionThread(action, file_path, output_path)
        self.encryption_thread.finished.connect(self.encryption_completed)
        self.encryption_thread.start()

    def encryption_completed(self, message):
        self.et_encrypt_btn.setEnabled(True)

    def init_metadata_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Handle Image Metadata")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        form_layout = QFormLayout()

        # Action Selection
        self.md_action = QComboBox()
        self.md_action.addItems(["Display Metadata", "Remove Metadata"])
        self.md_action.setToolTip("Choose whether to display or remove metadata from an image.")
        form_layout.addRow("Action:", self.md_action)

        # Select Image
        self.md_select_image_btn = QPushButton("📂 Select Image")
        self.md_select_image_btn.setToolTip("Choose the image file to work with.")
        self.md_select_image_btn.clicked.connect(self.select_metadata_image)
        form_layout.addRow("Image:", self.md_select_image_btn)

        self.md_image_label = QLabel("No image selected.")
        self.md_image_label.setStyleSheet("color: #555555;")
        form_layout.addRow("", self.md_image_label)

        # Select Output Path (For Removal)
        self.md_output_btn = QPushButton("📁 Select Output Path (For Removal)")
        self.md_output_btn.setToolTip("Choose where the cleaned image will be saved.")
        self.md_output_btn.clicked.connect(self.select_metadata_output)
        form_layout.addRow("Output Path:", self.md_output_btn)

        self.md_output_label = QLabel("No output path selected.")
        self.md_output_label.setStyleSheet("color: #555555;")
        form_layout.addRow("", self.md_output_label)

        layout.addLayout(form_layout)

        # Action Selection Changed
        self.md_action.currentTextChanged.connect(self.metadata_action_changed)

        # Process Button
        self.md_process_btn = QPushButton("🔧 Start")
        self.md_process_btn.setToolTip("Start displaying or removing metadata.")
        self.md_process_btn.clicked.connect(self.metadata_action_perform)
        self.md_process_btn.setEnabled(False)
        layout.addWidget(self.md_process_btn)

        # Result Display
        self.md_result = QTextEdit()
        self.md_result.setReadOnly(True)
        self.md_result.setPlaceholderText("Metadata will be displayed here.")
        layout.addWidget(self.md_result)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Metadata")

    def select_metadata_image(self):
        file, _ = QFileDialog.getOpenFileName(self, "Select Image")
        if file:
            self.md_image_label.setText(file)
            self.selected_metadata_image = file
            self.check_metadata_ready()

    def select_metadata_output(self):
        file, _ = QFileDialog.getSaveFileName(self, "Select Output File")
        if file:
            self.md_output_label.setText(file)
            self.selected_metadata_output = file
            self.check_metadata_ready()

    def check_metadata_ready(self):
        action = self.md_action.currentText()
        if action == "Display Metadata":
            if hasattr(self, 'selected_metadata_image'):
                self.md_process_btn.setEnabled(True)
        elif action == "Remove Metadata":
            if hasattr(self, 'selected_metadata_image') and hasattr(self, 'selected_metadata_output'):
                self.md_process_btn.setEnabled(True)

    def metadata_action_changed(self, text):
        if text == "Display Metadata":
            self.md_output_btn.setEnabled(False)
            self.md_output_label.setText("Not required.")
            self.check_metadata_ready()
        elif text == "Remove Metadata":
            self.md_output_btn.setEnabled(True)
            self.check_metadata_ready()

    def metadata_action_perform(self):
        action = self.md_action.currentText().lower().split()[0]
        image_path = self.selected_metadata_image
        output_path = getattr(self, 'selected_metadata_output', None)
        self.md_process_btn.setEnabled(False)
        self.md_result.clear()
        self.metadata_thread = MetadataThread(action, image_path, output_path)
        self.metadata_thread.finished.connect(self.metadata_completed)
        self.metadata_thread.start()

    def metadata_completed(self, result):
        if self.md_action.currentText() == "Display Metadata":
            self.md_result.setText(result)
        else:
            self.md_result.setText(result)
        self.md_process_btn.setEnabled(True)

    def init_batch_rename_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Batch Rename Files")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        form_layout = QFormLayout()

        # Select Directory
        self.br_select_dir_btn = QPushButton("📂 Select Directory")
        self.br_select_dir_btn.setToolTip("Choose the directory containing files to rename.")
        self.br_select_dir_btn.clicked.connect(self.select_rename_directory)
        form_layout.addRow("Directory:", self.br_select_dir_btn)

        self.br_directory_label = QLabel("No directory selected.")
        self.br_directory_label.setStyleSheet("color: #555555;")
        form_layout.addRow("", self.br_directory_label)

        # Renaming Pattern
        self.br_pattern = QLineEdit()
        self.br_pattern.setPlaceholderText("e.g., File_{index}{extension}")
        self.br_pattern.setToolTip("Use placeholders like {index}, {original_name}, and {extension} to define the renaming pattern.")
        form_layout.addRow("Renaming Pattern:", self.br_pattern)

        # Starting Index
        self.br_start_index = QSpinBox()
        self.br_start_index.setRange(1, 1000000)
        self.br_start_index.setValue(1)
        self.br_start_index.setToolTip("Define the starting number for indexing.")
        form_layout.addRow("Starting Index:", self.br_start_index)

        layout.addLayout(form_layout)

        # Rename Button
        self.br_rename_btn = QPushButton("🔧 Rename Files")
        self.br_rename_btn.setToolTip("Start renaming the selected files based on the defined pattern.")
        self.br_rename_btn.clicked.connect(self.batch_rename_action)
        self.br_rename_btn.setEnabled(False)
        layout.addWidget(self.br_rename_btn)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Batch Rename")

    def select_rename_directory(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Directory for Renaming")
        if directory:
            self.br_directory_label.setText(directory)
            self.selected_rename_directory = directory
            if self.br_pattern.text():
                self.br_rename_btn.setEnabled(True)

    def batch_rename_action(self):
        pattern = self.br_pattern.text()
        if not pattern:
            QMessageBox.warning(self, "Input Error", "Renaming pattern cannot be empty.")
            return
        start_index = self.br_start_index.value()
        self.br_rename_btn.setEnabled(False)
        self.rename_thread = BatchRenameThread(self.selected_rename_directory, pattern, start_index)
        self.rename_thread.finished.connect(self.rename_completed)
        self.rename_thread.start()

    def rename_completed(self):
        self.br_rename_btn.setEnabled(True)

    def init_reports_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Generate Reports")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        # Directory Size Report Section
        dsr_section = QWidget()
        dsr_layout = QVBoxLayout()
        dsr_label = QLabel("📊 Directory Size Report")
        dsr_label.setStyleSheet("font-size: 16px; font-weight: bold;")
        dsr_layout.addWidget(dsr_label)

        # Select Directory
        dsr_dir_layout = QHBoxLayout()
        self.r_select_dir_btn = QPushButton("📂 Select Directory")
        self.r_select_dir_btn.setToolTip("Choose the directory to analyze for size reporting.")
        self.r_select_dir_btn.clicked.connect(self.select_report_directory)
        dsr_dir_layout.addWidget(self.r_select_dir_btn)

        self.r_directory_label = QLabel("No directory selected.")
        self.r_directory_label.setStyleSheet("color: #555555;")
        dsr_dir_layout.addWidget(self.r_directory_label)

        dsr_layout.addLayout(dsr_dir_layout)

        # Report Path
        dsr_path_layout = QHBoxLayout()
        self.r_report_path = QLineEdit()
        self.r_report_path.setPlaceholderText("Enter report file path (e.g., report.txt)")
        self.r_report_path.setToolTip("Specify where the directory size report will be saved.")
        dsr_path_layout.addWidget(self.r_report_path)
        dsr_layout.addLayout(dsr_path_layout)

        # Generate Report Button
        self.r_generate_dsr_btn = QPushButton("📈 Generate Report")
        self.r_generate_dsr_btn.setToolTip("Create the directory size report based on the selected directory.")
        self.r_generate_dsr_btn.clicked.connect(self.generate_dsr_action)
        self.r_generate_dsr_btn.setEnabled(False)
        dsr_layout.addWidget(self.r_generate_dsr_btn)

        dsr_section.setLayout(dsr_layout)
        layout.addWidget(dsr_section)

        # Log Summary Report Section
        lsr_section = QWidget()
        lsr_layout = QVBoxLayout()
        lsr_label = QLabel("📝 Log Summary Report")
        lsr_label.setStyleSheet("font-size: 16px; font-weight: bold;")
        lsr_layout.addWidget(lsr_label)

        # Log Path
        lsr_log_layout = QHBoxLayout()
        self.r_log_path = QLineEdit()
        self.r_log_path.setPlaceholderText("Enter log file path (default 'file_manager.log')")
        self.r_log_path.setToolTip("Specify the path to the log file for generating the summary.")
        lsr_log_layout.addWidget(self.r_log_path)
        lsr_layout.addLayout(lsr_log_layout)

        # Summary Path
        lsr_summary_layout = QHBoxLayout()
        self.r_summary_path = QLineEdit()
        self.r_summary_path.setPlaceholderText("Enter summary file path (e.g., summary.txt)")
        self.r_summary_path.setToolTip("Specify where the log summary will be saved.")
        lsr_summary_layout.addWidget(self.r_summary_path)
        lsr_layout.addLayout(lsr_summary_layout)

        # Generate Summary Button
        self.r_generate_lsr_btn = QPushButton("📝 Generate Summary")
        self.r_generate_lsr_btn.setToolTip("Create a summary of the log file.")
        self.r_generate_lsr_btn.clicked.connect(self.generate_lsr_action)
        self.r_generate_lsr_btn.setEnabled(True)
        lsr_layout.addWidget(self.r_generate_lsr_btn)

        lsr_section.setLayout(lsr_layout)
        layout.addWidget(lsr_section)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Reports")

    def select_report_directory(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Directory for Size Report")
        if directory:
            self.r_directory_label.setText(directory)
            self.selected_report_directory = directory
            if self.r_report_path.text():
                self.r_generate_dsr_btn.setEnabled(True)

    def generate_dsr_action(self):
        directory = self.selected_report_directory
        report_path = self.r_report_path.text() if self.r_report_path.text() else "directory_size_report.txt"
        self.r_generate_dsr_btn.setEnabled(False)
        self.dsr_thread = DirectorySizeReportThread(directory, report_path)
        self.dsr_thread.finished.connect(self.dsr_completed)
        self.dsr_thread.start()

    def dsr_completed(self):
        self.r_generate_dsr_btn.setEnabled(True)

    def generate_lsr_action(self):
        log_path = self.r_log_path.text() if self.r_log_path.text() else 'file_manager.log'
        summary_path = self.r_summary_path.text() if self.r_summary_path.text() else 'log_summary.txt'
        self.r_generate_lsr_btn.setEnabled(False)
        self.lsr_thread = LogSummaryThread(log_path, summary_path)
        self.lsr_thread.finished.connect(self.lsr_completed)
        self.lsr_thread.start()

    def lsr_completed(self):
        self.r_generate_lsr_btn.setEnabled(True)

    def init_monitor_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Real-Time Directory Monitoring")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        # Select Directory Button and Label
        dir_layout = QHBoxLayout()
        self.m_select_dir_btn = QPushButton("📂 Select Directory to Monitor")
        self.m_select_dir_btn.setToolTip("Choose the directory to monitor for new files.")
        self.m_select_dir_btn.clicked.connect(self.select_monitor_directory)
        dir_layout.addWidget(self.m_select_dir_btn)

        self.m_directory_label = QLabel("No directory selected.")
        self.m_directory_label.setStyleSheet("color: #555555;")
        dir_layout.addWidget(self.m_directory_label)

        layout.addLayout(dir_layout)

        # Start Monitoring Button
        self.m_start_btn = QPushButton("🚀 Start Monitoring")
        self.m_start_btn.setToolTip("Begin real-time monitoring of the selected directory.")
        self.m_start_btn.clicked.connect(self.start_monitoring)
        self.m_start_btn.setEnabled(False)
        layout.addWidget(self.m_start_btn)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Monitor")

    def select_monitor_directory(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Directory to Monitor")
        if directory:
            self.m_directory_label.setText(directory)
            self.selected_monitor_directory = directory
            self.m_start_btn.setEnabled(True)

    def start_monitoring(self):
        self.m_start_btn.setEnabled(False)
        self.monitor_thread = QThread()
        self.watcher = Watcher(self.selected_monitor_directory)
        self.watcher.moveToThread(self.monitor_thread)
        self.monitor_thread.started.connect(self.watcher.run)
        self.monitor_thread.start()

    def init_cache_cleaner_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Cache and Temporary Files Cleaner")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        info_lbl = QLabel("Clean cache and temporary files to free up space and enhance system performance.")
        info_lbl.setWordWrap(True)
        layout.addWidget(info_lbl)

        # Platform-Specific Cache Directories
        cache_dirs = self.get_cache_directories()
        self.cache_list_widget = QListWidget()
        for dir_path in cache_dirs:
            item = QListWidgetItem(dir_path)
            item.setCheckState(Qt.Unchecked)
            self.cache_list_widget.addItem(item)
        self.cache_list_widget.setToolTip("Select the cache directories you want to clean.")
        layout.addWidget(self.cache_list_widget)

        # Select All and Deselect All Buttons
        select_buttons_layout = QHBoxLayout()
        self.cc_select_all_btn = QPushButton("✅ Select All")
        self.cc_select_all_btn.setToolTip("Select all cache directories.")
        self.cc_select_all_btn.clicked.connect(self.select_all_caches)
        select_buttons_layout.addWidget(self.cc_select_all_btn)

        self.cc_deselect_all_btn = QPushButton("❌ Deselect All")
        self.cc_deselect_all_btn.setToolTip("Deselect all cache directories.")
        self.cc_deselect_all_btn.clicked.connect(self.deselect_all_caches)
        select_buttons_layout.addWidget(self.cc_deselect_all_btn)

        layout.addLayout(select_buttons_layout)

        # Clean Button
        self.cc_clean_btn = QPushButton("🧹 Clean Selected Directories")
        self.cc_clean_btn.setToolTip("Start cleaning the selected cache directories.")
        self.cc_clean_btn.clicked.connect(self.clean_cache_dirs)
        self.cc_clean_btn.setEnabled(False)
        layout.addWidget(self.cc_clean_btn)

        # Progress Bar
        self.cc_progress = QProgressBar()
        layout.addWidget(self.cc_progress)

        # Connect selection changes to enable/disable clean button
        self.cache_list_widget.itemChanged.connect(self.update_clean_button_state)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Cache Cleaner")

    def get_cache_directories(self):
        dirs = []
        if sys.platform.startswith('win'):
            # Windows Cache Directories
            dirs.extend([
                os.getenv('LOCALAPPDATA', '') + '\\Temp',
                os.getenv('TEMP', ''),
                os.path.join(os.getenv('LOCALAPPDATA', ''), 'Google', 'Chrome', 'User Data', 'Default', 'Cache'),
                os.path.join(os.getenv('LOCALAPPDATA', ''), 'Mozilla', 'Firefox', 'Profiles')
            ])
        elif sys.platform.startswith('darwin'):
            # macOS Cache Directories
            dirs.extend([
                os.path.expanduser('~/Library/Caches'),
                '/private/var/tmp',
                '/private/tmp'
            ])
        else:
            # Linux Cache Directories
            dirs.extend([
                os.path.expanduser('~/.cache'),
                '/tmp',
                '/var/tmp'
            ])
        # Remove any empty paths
        dirs = [d for d in dirs if d and os.path.exists(d)]
        return dirs

    def select_all_caches(self):
        for index in range(self.cache_list_widget.count()):
            item = self.cache_list_widget.item(index)
            item.setCheckState(Qt.Checked)

    def deselect_all_caches(self):
        for index in range(self.cache_list_widget.count()):
            item = self.cache_list_widget.item(index)
            item.setCheckState(Qt.Unchecked)

    def update_clean_button_state(self, item):
        any_checked = any(
            self.cache_list_widget.item(i).checkState() == Qt.Checked
            for i in range(self.cache_list_widget.count())
        )
        self.cc_clean_btn.setEnabled(any_checked)

    def clean_cache_dirs(self):
        selected_dirs = [
            self.cache_list_widget.item(i).text()
            for i in range(self.cache_list_widget.count())
            if self.cache_list_widget.item(i).checkState() == Qt.Checked
        ]
        if not selected_dirs:
            QMessageBox.warning(self, "No Directories Selected", "Please select at least one directory to clean.")
            return
        confirm = QMessageBox.question(self, "Confirm Cleaning",
                                       "Are you sure you want to delete all files in the selected directories?\nThis action cannot be undone.",
                                       QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if confirm != QMessageBox.Yes:
            return
        self.cc_clean_btn.setEnabled(False)
        self.cc_progress.setValue(0)
        self.cache_thread = CacheCleanerThread(selected_dirs)
        self.cache_thread.progress.connect(self.cc_progress.setValue)
        self.cache_thread.finished.connect(self.cache_cleaning_completed)
        self.cache_thread.start()

    def cache_cleaning_completed(self):
        self.cc_clean_btn.setEnabled(True)
        self.cc_progress.setValue(100)
        QMessageBox.information(self, "Cleaning Completed", "Selected cache directories have been cleaned.")
        logging.info("Cache cleaning completed.")
        notification.notify(
            title="Cache Cleaning Completed",
            message="Selected cache directories have been cleaned.",
            timeout=5
        )

    def init_settings_tab(self):
        tab = QWidget()
        layout = QVBoxLayout()

        lbl = QLabel("Settings")
        lbl.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(lbl)

        form_layout = QFormLayout()

        # Backup Directory
        self.s_backup_dir_btn = QPushButton("📁 Select Backup Directory")
        self.s_backup_dir_btn.setToolTip("Choose where backups of deleted or moved files will be stored.")
        self.s_backup_dir_btn.clicked.connect(self.select_backup_directory)
        form_layout.addRow("Backup Directory:", self.s_backup_dir_btn)

        self.s_backup_dir_label = QLabel(config.get("backup_directory", "Backup"))
        self.s_backup_dir_label.setStyleSheet("color: #555555;")
        form_layout.addRow("", self.s_backup_dir_label)

        # Email Notifications
        self.s_email_enabled = QCheckBox("Enable Email Notifications")
        self.s_email_enabled.setChecked(config.get("email_notifications", {}).get("enabled", False))
        self.s_email_enabled.setToolTip("Enable or disable email notifications for various operations.")
        form_layout.addRow(self.s_email_enabled)

        self.s_smtp_server = QLineEdit(config.get("email_notifications", {}).get("smtp_server", ""))
        self.s_smtp_server.setToolTip("Enter the SMTP server address (e.g., smtp.gmail.com).")
        form_layout.addRow("SMTP Server:", self.s_smtp_server)

        self.s_smtp_port = QSpinBox()
        self.s_smtp_port.setRange(1, 65535)
        self.s_smtp_port.setValue(config.get("email_notifications", {}).get("smtp_port", 587))
        self.s_smtp_port.setToolTip("Enter the SMTP server port (e.g., 587 for TLS).")
        form_layout.addRow("SMTP Port:", self.s_smtp_port)

        self.s_sender_email = QLineEdit(config.get("email_notifications", {}).get("sender_email", ""))
        self.s_sender_email.setToolTip("Enter the sender's email address.")
        form_layout.addRow("Sender Email:", self.s_sender_email)

        self.s_sender_password = QLineEdit()
        self.s_sender_password.setEchoMode(QLineEdit.Password)
        self.s_sender_password.setToolTip("Enter the sender's email password.")
        form_layout.addRow("Sender Password:", self.s_sender_password)

        self.s_receiver_email = QLineEdit(config.get("email_notifications", {}).get("receiver_email", ""))
        self.s_receiver_email.setToolTip("Enter the receiver's email address.")
        form_layout.addRow("Receiver Email:", self.s_receiver_email)

        layout.addLayout(form_layout)

        # Save Settings Button
        self.s_save_btn = QPushButton("💾 Save Settings")
        self.s_save_btn.setToolTip("Save all the configured settings.")
        self.s_save_btn.clicked.connect(self.save_settings)
        layout.addWidget(self.s_save_btn)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "Settings")

    def select_backup_directory(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Backup Directory")
        if directory:
            self.s_backup_dir_label.setText(directory)
            config['backup_directory'] = directory

    def save_settings(self):
        config['email_notifications']['enabled'] = self.s_email_enabled.isChecked()
        config['email_notifications']['smtp_server'] = self.s_smtp_server.text()
        config['email_notifications']['smtp_port'] = self.s_smtp_port.value()
        config['email_notifications']['sender_email'] = self.s_sender_email.text()
        config['email_notifications']['sender_password'] = self.s_sender_password.text()
        config['email_notifications']['receiver_email'] = self.s_receiver_email.text()
        with open(CONFIG_FILE, 'w') as f:
            json.dump(config, f, indent=4)
        QMessageBox.information(self, "Settings Saved", "Settings have been saved successfully.")
        logging.info("Settings updated and saved.")
        notification.notify(
            title="Settings Saved",
            message="All settings have been saved successfully.",
            timeout=5
        )

# ===========================
# Main Execution
# ===========================

if __name__ == '__main__':
    app = QApplication(sys.argv)
    gui = FileManagerGUI()
    gui.show()
    sys.exit(app.exec_())
